# Making Business Decisions to Maximise Sales of a Digital Media Store

We will be using the Chinook Database, which contains data from an imaginary digital media store Chinook. The database is a modified version of a sample database from the iTunes Library. The Chinook Database contains several tables which stores information of customers, invoices, artists, albums, and tracks from its store.

The original unmodified file can be found on [GitHub](https://github.com/lerocha/chinook-database#:~:text=Chinook%20Database,single%20and%20multiple%20database%20servers.), where I cite how they created the database as shown below:

> Media related data was created using real data from an iTunes Library. It is possible for you to use your own iTunes Library to generate the SQL scripts, see instructions below. Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.). Sales information is auto generated using random data for a four year period.

The goal of this project is to examine how we can make appropriate business decisions to potentially maximise our sales (in terms of songs) in Chinook.

Let's begin by connecting our Jupyter Notebook to the database file.

### Connecting to Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

#### Obtaining a List of all Tables and Views in the Database

A view is similar to a table, and can be created from a table or other views. They are based off tables and don't actually contain data, but are rather utilising the data from the main table they are created from. If that particular table changes, the view would change as well.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


I have attached a picture below to show how the tables are related to one another:

<img src="chinook-schema.svg">

In [3]:
%%sql
SELECT DISTINCT country
FROM customer;

 * sqlite:///chinook.db
Done.


country
Brazil
Germany
Canada
Norway
Czech Republic
Austria
Belgium
Denmark
USA
Portugal


### Data Exploration

#### Finding the Top Selling Genres

A great starting point would be great to see which genres (of songs) sell the most in the USA. We shall do this by calculating the number of tracks of each genre sold in the USA, along with the percentage each genre comprises.

In [4]:
%%sql

WITH track_w_genre AS
     (SELECT t.*,
             g.name genre_name
      FROM track t
      INNER JOIN genre g ON t.genre_id = g.genre_id
     )
    
SELECT * FROM track_w_genre
LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price,genre_name
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,Rock
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Rock
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99,Rock
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99,Rock
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Rock


In [5]:
%%sql

WITH track_w_genre AS
     (SELECT t.*,
             g.name genre_name
      FROM track t
      INNER JOIN genre g ON t.genre_id = g.genre_id
     ),
     total_songs_sold AS
    (SELECT
        SUM(il.quantity) total_sold
     FROM track_w_genre twg
     INNER JOIN invoice_line il ON il.track_id = twg.track_id
    )

SELECT 
    twg.genre_name genre_name,
    SUM(il.quantity) tracks_sold,
    ROUND(CAST(SUM(il.quantity) AS Float)*100/(SELECT * FROM total_songs_sold), 2) percentage_sold
FROM invoice_line il
INNER JOIN track_w_genre twg ON twg.track_id = il.track_id
GROUP BY twg.genre_name
ORDER BY tracks_sold DESC;

 * sqlite:///chinook.db
Done.


genre_name,tracks_sold,percentage_sold
Rock,2635,55.39
Metal,619,13.01
Alternative & Punk,492,10.34
Latin,167,3.51
R&B/Soul,159,3.34
Blues,124,2.61
Jazz,121,2.54
Alternative,117,2.46
Easy Listening,74,1.56
Pop,63,1.32


We see that `Rock` is the top genre in the iTunes USA Library, followed by `Metal` and `Alternative & Punk`. Let's have a look at the top selling songs in the USA to confirm that it isn't just one or two songs skewing towards a particular genre, in terms of the number sold.

In [6]:
%%sql

WITH track_w_genre AS
     (SELECT t.*,
             g.name genre_name
      FROM track t
      INNER JOIN genre g ON t.genre_id = g.genre_id
     )

SELECT 
    twg.name track_name,
    twg.genre_name genre_name,
    SUM(il.quantity) tracks_sold
FROM invoice_line il
INNER JOIN track_w_genre twg ON twg.track_id = il.track_id
GROUP BY twg.track_id
ORDER BY tracks_sold DESC
LIMIT 30;

 * sqlite:///chinook.db
Done.


track_name,genre_name,tracks_sold
War Pigs,Alternative,31
Are You Experienced?,Rock,14
Highway Chile,Rock,14
Put The Finger On You,Rock,13
Third Stone From The Sun,Rock,13
Hey Joe,Rock,13
Love Or Confusion,Rock,12
Radio/Video,Metal,12
Dead And Broken,Metal,11
Foxy Lady,Rock,11


This suggests that we are on the right track, and that we should be looking towards releasing albums where artists specialise in top selling genres, such as `Rock`, `Metal` and `Alternative & Punk`.

#### Examining Employee Sales Performance

In Chinook, each customer is assigned to a sales support agent within the company when they first make a purchase. In order to ascertain the performance of each sales support agent, we would like to look at how much money each of them made for the company.

In [7]:
%%sql

WITH em_cust_inv AS
    (
     SELECT e1.*, e2.first_name || " " || e2.last_name direct_superior, c.*, i.*
     FROM employee e1
     LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
     LEFT JOIN customer c ON c.support_rep_id = e1.employee_id
     INNER JOIN invoice i ON c.customer_id = i.customer_id
    )
    
SELECT e.first_name || " " || e.last_name employee_name,
       e.direct_superior,
       e.hire_date - e.birthdate age,
       e.hire_date,
       e.country,
       ROUND(SUM(e.total), 2) total_revenue
FROM em_cust_inv e
WHERE e.title = "Sales Support Agent"
GROUP BY employee_id
ORDER BY total_revenue DESC;

 * sqlite:///chinook.db
Done.


employee_name,direct_superior,age,hire_date,country,total_revenue
Jane Peacock,Nancy Edwards,44,2017-04-01 00:00:00,Canada,1731.51
Margaret Park,Nancy Edwards,70,2017-05-03 00:00:00,Canada,1584.0
Steve Johnson,Nancy Edwards,52,2017-10-17 00:00:00,Canada,1393.92


Although it is observed that Steve makes slightly lesser than Margaret and Jane, the latter were hired months earlier than Steve. Therefore, the total revenue alone does not necessarily make a good indicator of the Sales Support Agents' performance.

#### Analysing Sales in Different Countries

We would like to observe Chinook's sales performance in each country. Therefore, we need to gather information about each country's:

* total number of customers
* total value made by sales
* average value of sales per customer
* average value of each invoice order

In [8]:
%%sql

WITH country_or_other AS
        (SELECT
             CASE
                 WHEN
                     (SELECT COUNT(*)
                     FROM customer
                     WHERE country = c.country) = 1 THEN "Other"
                 ELSE c.country
             END AS country,
             c.customer_id,
             il.*
         FROM invoice_line il
         INNER JOIN invoice i ON il.invoice_id = i.invoice_id
         INNER JOIN customer c ON i.customer_id = c.customer_id
        ),
    country_each_val AS
        (SELECT country,
                COUNT(DISTINCT customer_id) num_customers,
                ROUND(SUM(unit_price), 2) total_value,
                ROUND(SUM(unit_price)/COUNT(DISTINCT customer_id), 2) avg_per_cust,
                ROUND(SUM(unit_price)/COUNT(DISTINCT invoice_id), 2) avg_per_inv,
                CASE
                    WHEN country = "Other" THEN 1
                    ELSE 0
                END AS country_sort
         FROM country_or_other
         GROUP BY country
         ORDER BY country_sort ASC, total_value DESC
        )
        
SELECT country,
       num_customers,
       total_value,
       avg_per_cust,
       avg_per_inv
FROM country_each_val;


 * sqlite:///chinook.db
Done.


country,num_customers,total_value,avg_per_cust,avg_per_inv
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


Besides the USA, other countries which show promise to invest marketing efforts in are Czech Republic, Portugal and India, since they have a high average spending per customer. However, we need to be careful because the sample size is low in this case; there are not even 5 customers from each country that is represented here. 